<a href="https://colab.research.google.com/github/chetwyndchannel/My-projects/blob/main/MSDS_599_Final_PRoject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abstract

**Goal**

The Goal of this analysis is to create an efficient price predictor using regression models. We hope that through our analysis for airbnb property owners to better adjust their rate based on their properties attributes. 


**Data Overview**

The data set that we will be utilizing for this analysis is from Airbnb’s New York metropolitan area. The shape of our data is 86009 thousand rows and 46 columns. It comprises 46 different variables; we will only include 36 of these variables which have been chosen based on their ability to predict the price of an Airbnb property in New york. In our analysis we had 5 different price predictors to choose from: Published Nightly rate, Average daily rate, Published Monthly rate, Published Weekly rate and Annual Revenue. We decided that Average Daily Rate(ADR) and Published Nightly Rate (PNR), would be the best predictors for our models.


The ADR variable had 22617 missing values (26%), however instead of dropping these NAs we decided to feature engineer a value to replace them. First we identified the top recurring neighborhoods which accounted for 60% of the missing values in ADR: Williamsburg; Harlem; Brooklyn; Midtown;  Bushwick; East Village;  Upper West Side; Upper East Side; Crown Heights; Lower East Side; Astoria; Washington Heights. 


Then we took the mode of these values and the respective mean. The result we got from these calculations was 87.5, each NA value within ADR was replaced with this value. 
After this process we decided to drop the rest of the null values resulting in a sample size of 10038, which gives an additional 2471 samples from just dropping all null values (see: data6 vs data5).  From this process we create the night_data dataset (relevant columns excluding ADR) and the ADR_data dataset (relevant columns excluding Published Nightly Rate), for their respective prediction models. 


Next we tested which dependent variable performed the best in a standard linear regression. The results were that Published Nightly Rate ($R^2=0.723$) out performed Average Daily Price ($R^2=0.635$). Our reasoning behind this is that perhaps we could have used a better method for feature engineering to get more similar results. 

We ran the rest of the models using the PNR as the main dependent variable.

**Summary**

For the analysis of this data set, we have chosen to utilize 6 different supervised machine learning models to predict the price of an Airbnb property in New York. For our first model we have chosen a principal component analysis as a dimension reduction technique. From the PCA model we have been able to reduce the amount of linear relationships needed to represent our model down to 3 principal components, however this greatly reduces the predictive power of our models. We theorized that this might  be due to the fact that  PCA does not take into account classes of information, it purely looks at the variance of each feature.

 The second model we use an *OLS Regression* technique to help understand the causal relationship between the features of a property on the published nightly rate.
The R Squared from this model is 0.72 ~ 72.3%.  

Our third model utilizes *Ridge Regression* to combat the issue of multicollinearity. The R squared from this model is .72 ~ 72.3%. In the fourth model we use a *Lasso Regression* to help shrink the possibility of overfitting. The R squared from this model is 0.723 ~ 72.3%. 

The fourth model is a *Random Forest regressor*. This technique allows us to fit a number of classifying decisions on various sub-samples of the data set using averaging to improve the accuracy and control for overfitting. The R squared for this model is 0.732 ~ 73.2%. 

In the fifth model we use a *Gradient boosting model*. This model allows for weaker signals to show throughout the model, this also shrinks the data set controlling for overfitting. The R squared for this model is 0.58~58%.

 The sixth model we have chosen is *Bagging* in order to help with bias-variance-trade-off. The R squared for this model is 0.719 ~71%. 

After utilizing the 6 different  supervised machine learning models we can conclude that the Random Forest regressor is the most efficient model for price prediction of an airbnb property in New York. Its ability to combine ensemble learning with a decision tree framework to create multiple randomly drawn decision trees from the data allowed for the model to be flexible enough to accurately predict Nightly rate. Information that shed light on the price setting controls set by owners and the different types of discounts given by owners based on seasonality. This information would help us better understand price setting for airbnb owners  leading to a more accurate price prediction model.


# Data Exploration/Cleaning


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression as ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/airbnb_property/New York-New York_Property_2017-03-01.csv')
data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Property ID,Host ID,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,...,Count Reservation Days LTM,Count Available Days LTM,Count Blocked Days LTM,Number of Photos,Business Ready,Instantbook Enabled,Listing URL,Listing Main Image URL,Latitude,Longitude
0,105.0,500.0,Trendy Times Square Loft,Apartment,Private room,2008-06-28,2016-03-18,United States,New York,New York,...,0.0,366.0,0.0,25.0,False,No,https://www.airbnb.com/rooms/105,https://a0.muscache.com/im/pictures/59366/b695...,40.762099,-73.996022
1,795.0,1061.0,Sky Chelsea 3 bdr,Apartment,Entire home/apt,2008-07-25,2016-03-02,United States,New York,New York,...,0.0,366.0,0.0,22.0,False,No,https://www.airbnb.com/rooms/795,https://a2.muscache.com/im/pictures/a849796f-a...,40.746257,-74.007022
2,2515.0,2758.0,Stay at Chez Chic budget room #1,Apartment,Private room,2008-09-05,2017-03-01,United States,New York,New York,...,239.0,106.0,21.0,11.0,False,No,https://www.airbnb.com/rooms/2515,https://a0.muscache.com/im/pictures/88569/eebe...,40.799205,-73.953676
3,2534.0,2782.0,"The Nest: Cozy, Quiet, Clean, Sunny",Apartment,Entire home/apt,2008-09-06,2016-02-19,United States,New York,New York,...,103.0,8.0,255.0,16.0,False,No,https://www.airbnb.com/rooms/2534,https://a2.muscache.com/im/pictures/7a147a6c-e...,40.762059,-73.961365
4,2539.0,2787.0,Clean & quiet apt home by the park,Apartment,Private room,2008-09-07,2016-11-19,United States,New York,New York,...,38.0,328.0,0.0,12.0,False,No,https://www.airbnb.com/rooms/2539,https://a2.muscache.com/im/pictures/3949d073-a...,40.647486,-73.972370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86004,16905634.0,1009903.0,Comfort and coziness,Apartment,Entire home/apt,2017-01-23,2017-02-05,United States,New York,New York,...,0.0,31.0,0.0,7.0,False,Yes,https://www.airbnb.com/rooms/16905634,https://a0.muscache.com/im/pictures/23fad023-0...,40.735170,-74.005583
86005,16908288.0,24120303.0,Fresh Shared Space,Apartment,Shared room,2017-01-23,2017-02-03,United States,New York,New York,...,0.0,31.0,0.0,8.0,False,No,https://www.airbnb.com/rooms/16908288,https://a0.muscache.com/im/pictures/e9073702-1...,40.706625,-73.949328
86006,16909197.0,24120303.0,Fun Cozy Environment,Apartment,Shared room,2017-01-23,2017-02-03,United States,New York,New York,...,0.0,28.0,3.0,6.0,False,No,https://www.airbnb.com/rooms/16909197,https://a0.muscache.com/im/pictures/5d626c2a-3...,40.707130,-73.949391
86007,16909522.0,24120303.0,Chilled Shared Space,Apartment,Shared room,2017-01-23,2017-02-02,United States,New York,New York,...,0.0,31.0,0.0,9.0,False,No,https://www.airbnb.com/rooms/16909522,https://a0.muscache.com/im/pictures/6db0080f-8...,40.707067,-73.949980


In [ ]:
from datetime import datetime
#datetime.strptime(data['Last Scraped Date'], '%m/%d/%y %H:%M:%S')
pd.to_datetime(data ['Created Date']).min()


Timestamp('2008-06-28 00:00:00')

# Data Engineering

In [ ]:
data.dtypes

In [ ]:
data.shape

In [ ]:
(data.isna().sum())/len(data)

In [ ]:
data.isna().sum()

In [ ]:
#Variables Dropped
#One value variables: City, Country 
#Unneeded variables: Listing URL, Listing Main Image URL, Latitude, Longitude, Metropolitan Statistical Area, state
#Checkout/Check-in Time variables had too many NA values, and had too many character variations making it difficult to format .
print('City:', data.City.unique())
print('Country:',data.Country.unique())
print('Metropolitan Statistical Area:', data['Metropolitan Statistical Area'].unique())
print('Check-in Time:', data['Check-in Time'].unique()[:4])


In [ ]:
#Dropping variables
data2=data.drop(columns=['City', 'State',"Country", "Listing URL", "Listing Main Image URL", "Latitude", "Longitude", "Checkout Time", "Check-in Time","Metropolitan Statistical Area"])
data2.shape


In [ ]:
data2

Originally we wanted to use Average Daily Rate as our price predictor as it gives the most accurate representation of a price over time.

However it has many NAs (26%), instead of dropping them we decided to engineer an appropriate value to replace the missing values. 


In [ ]:
print(data2["Average Daily Rate"].isna().sum())
print('Average Daily Rate percentage:',(data2["Average Daily Rate"].isna().sum())/len(data2["Average Daily Rate"]))

In [ ]:
#Finding only the NA Average Daily Rate (ADR)
adr_null=data2[data2['Average Daily Rate'].isnull()]

#Grouping by neighborhoods
len(adr_null['Neighborhood']) *0.6
sum(adr_null['Neighborhood'].value_counts().head(12))
adr_null['Neighborhood'].value_counts().head(12)

#top 12 neighborhoods
top_12 =['Williamsburg', 'Harlem', 'Brooklyn', 'Midtown', 'Bushwick', 'East Village', 'Upper West Side', 'Upper East Side', 'Crown Heights', 'Lower East Side', 'Astoria', 'Washington Heights']
data3=data2[data2['Neighborhood'].isin(['Williamsburg', 'Harlem', 'Brooklyn', 'Midtown', 'Bushwick', 'East Village', 'Upper West Side', 'Upper East Side', 'Crown Heights', 'Lower East Side', 'Astoria', 'Washington Heights'])]

#finding the mean of the mode 
data4 =data3.groupby(by=['Neighborhood'], as_index=False)['Average Daily Rate'].agg(pd.Series.mode)
print('Computed ADR:', data4["Average Daily Rate"].mean())
data4


Replacing the null values in ADR

- ADR has 22617 null values
- The top 12 reccuring neighborhoods made up 60% of the null values
- We took the mode of these 12 neighborhoods from the original dataset and the mean of their mode.
- replaced all null values with this result 87.5


##Cleaning


Engineering categories and dummies


In [ ]:
#Replacing dummy variables
print('Business Ready:', data2['Business Ready'].unique())
print('Instantbook Enabled:', data2['Instantbook Enabled'].unique())
print('Superhost:', data2['Superhost'].unique())

data5=pd.get_dummies(data2, columns=['Business Ready', 'Instantbook Enabled', 'Superhost' ], drop_first=True)
data5=data5.rename(columns={'Business Ready_True':'Business Ready','Superhost_True':'Superhost', 'Instantbook Enabled_Yes':'Instantbook Enabled'})

In [ ]:
data5

Replacing null ADR with 87.5

In [ ]:
#Replacing NAs with mode of the mean of top 12 neighborhood Average Daily Rate
data5['Average Daily Rate']=np.where(data2['Average Daily Rate'].isnull(), 87.5, data2['Average Daily Rate'])

#Creating variable to keep track of engineered results
data5['check_adr']=np.where(data5["Average Daily Rate"]==87.5, 1, 0)
data5.columns
data5.isna().sum()

In [ ]:
#We can see that by making adjustments to the ADR we're able to save  2471 data points
print(len(data.dropna()))
print(len(data5.dropna()))
print(len(data5.dropna())-len(data.dropna()))

In [ ]:
data6=data5.dropna()

In [ ]:
# Creating categories
print('Listing Type:', data5['Listing Type'].unique())
print('Property Type:', data5['Property Type'].unique())
print('Cancellation Policy:', data5['Cancellation Policy'].unique())

data6['Listing Type'] = pd.factorize(data6['Listing Type'])[0] + 1
data6['Property Type'] = pd.factorize(data6['Property Type'])[0] + 1
data6['Cancellation Policy']=pd.factorize(data6['Property Type'])[0] + 1

Two different datasets each with a different dependent price variable:

- ADR_data: views the data with average daily rate as the dependent variable

- Night_data views the data with Published Nightly rate as the dependent variable

In [ ]:
#Category:"Listing Title" , "Property Type", "Listing Type"
#dummies: 'Superhost', 'Business Ready', "Instant book Enabled"
#Price variables: "Average Daily Rate", "Annual Revenue LTM", "Number of Bookings LTM", "Published Nightly Rate", "Published Monthly Rate", "Published Weekly Rate"

#Looking at the difference between nightly rate and average daily rate
a=data2[["Published Nightly Rate", 'Average Daily Rate']].dropna()
ab= a['Published Nightly Rate'] - a['Average Daily Rate']
ab. mean()



In [ ]:
#Data with Published nightly rate as dependent variable
#Removing other price variables
Night_data=data6.drop(columns=['Property ID','Neighborhood','Host ID', 'Listing Title', 'Created Date', 'Last Scraped Date', 'Zipcode', 'Annual Revenue LTM', 'Calendar Last Updated', 'Response Time (min)', 'Average Daily Rate', 'Published Monthly Rate','Published Weekly Rate'])
Night_data


In [ ]:
#Data with Average daily Rate
#Removing other price variables
ADR_data=data6.drop(columns=['Property ID','Neighborhood','Host ID', 'Listing Title', 'Created Date', 'Last Scraped Date', 'Zipcode', 'Annual Revenue LTM', 'Calendar Last Updated', 'Response Time (min)', 'Published Nightly Rate', 'Published Monthly Rate','Published Weekly Rate'])
ADR_data

# Regressions

Average Daily Rate (ADR)

In [ ]:
X=ADR_data.drop(['Average Daily Rate'],axis=1)
y=ADR_data['Average Daily Rate']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=5)

In [ ]:
from sklearn.preprocessing import StandardScaler
X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit_transform(X_test)

Published Nightly Rate (PNR)

In [ ]:
X1= Night_data.drop(columns=['Published Nightly Rate'])
y1= Night_data['Published Nightly Rate']

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,test_size=0.3, random_state=5)
X1_train_scaled = StandardScaler().fit_transform(X1_train)
X1_test_scaled = StandardScaler().fit_transform(X1_test)

In [ ]:
#Feature importance 
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesRegressor
lab_enc = preprocessing.LabelEncoder()

feature_model = ExtraTreesRegressor(n_estimators=50)
feature_model.fit(X1_train,lab_enc.fit_transform(y1_train))

plt.figure(figsize=(7,7))
feat_importances = pd.Series(feature_model.feature_importances_, index=Night_data.iloc[:,:-1].columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

## PCA

###PCA Average Daily Rate

In [ ]:
ADR_data

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
X_train_scaled.shape

In [ ]:
pca = PCA(23).fit(X_train_scaled)

In [ ]:
pca_tr = pca.transform(X_train_scaled)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

In [ ]:
crvalres=[]
num_pcs=[1,2,3,4,5,6,7,8,9,10]
for a in num_pcs:
    reg = LinearRegression()
    crvalreg = cross_validate(reg, pca_tr[:,0:a], y_train, cv=5, scoring='r2')
    crvalres.append(crvalreg['test_score'].mean())

In [ ]:
crvalres

In [ ]:
np.argmax(crvalres)

In [ ]:
plt.plot(range(1,11),crvalres)

In [ ]:
for col,val in zip(X.columns,pca.components_[0]):
    print(col,":",val)

In [ ]:
for col,val in zip(X.columns,pca.components_[1]):
    print(col,":",val)

### PCA Published Nightly Rate

In [ ]:
Night_data

In [ ]:
X1_train_scaled.shape

In [ ]:
pca = PCA(23).fit(X1_train_scaled)

In [ ]:
pca_tr = pca.transform(X1_train_scaled)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
crvalres=[]
num_pcs=[1,2,3,4,5,6,7,8,9,10]
for a in num_pcs:
    reg = LinearRegression()
    crvalreg = cross_validate(reg, pca_tr[:,0:a], y1_train, cv=5, scoring='r2')
    crvalres.append(crvalreg['test_score'].mean())

In [ ]:
crvalres

In [ ]:
plt.plot(range(1,11),crvalres)

In [ ]:
np.argmax(crvalres)

In [ ]:
for col, val in zip(X1.columns,pca.components_[0]):
  print(col,':',val)

## OLS Regression

AS shown, Published Nightly Rate has a better predictive result than Average Daily Rate.

- Nightly Rate: $R^2$= 0.724
- Average Daily Rate: $R^2$ = 0.636

For this reason we chose to run the rest of the models only using Published Nightly Rate.

In [ ]:
#OlS for ADR
from sklearn import linear_model
reg=linear_model.LinearRegression()
adjusted=reg.fit(X_train, y_train)
adjusted.score(X_test,y_test)

In [ ]:
#OLS for Published Nightly Rate
from sklearn import linear_model
reg=linear_model.LinearRegression()
adjusted=reg.fit(X1_train, y1_train)
adjusted.score(X1_test,y1_test)

In [ ]:
#Testing for multicollinearty 
#Using a threshold of >10, we assume there's no multicollinearity between the independent variables.

import statsmodels.api as sm
x_temp = sm.add_constant(X1_train)

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x_temp.values, i) for i in range(x_temp.values.shape[1])]
vif["features"] = x_temp.columns
print(vif)

In [ ]:
#Cleaning Fee has strong heteroskedasticity
plt.hist(Night_data['Cleaning Fee'])

In [ ]:
Liner_Regression_results=pd.DataFrame({'Models':['LinerRegression'],
'R2': [adjusted.score(X1_test,y1_test)],
'MSE': [mean_squared_error(adjusted.predict(X1_test), y1_test)],
'MAE': [mean_absolute_error(adjusted.predict(X1_test),y1_test)]})

## Rigde Regression 

Rigde Model: 
- $R^2=0.723$
- $MSE=7136$
- $MAE =53.56$



In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
np.linspace(5,-3,100)

In [ ]:
lambdas = 10**np.linspace(5,-3,100)
lambdas

In [ ]:
ridgecv = RidgeCV(alphas = lambdas, scoring = 'neg_mean_squared_error', normalize = True,cv=5)
ridgecv.fit(X1_train, y1_train)
ridgecv.alpha_

In [ ]:
ridgecv.alpha_

In [ ]:
ridgemod = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridgemod.fit(X1_train, y1_train)

In [ ]:
RidgeCV_R2 = ridgemod.score(X1_test,y1_test).round(3)
RidgeCV_MSE = mean_squared_error(ridgemod.predict(X1_test),y1_test).round(3)
RidgeCV_MAE = mean_absolute_error(ridgemod.predict(X1_test),y1_test).round(3)
print('R^2:', RidgeCV_R2)
print('MSE:', RidgeCV_MSE)
print('MAE:', RidgeCV_MAE)

In [ ]:
Ridge_Regression_results=pd.DataFrame({'Models':['RidgeRegression_CV'],
'R2' : [ridgemod.score(X1_test,y1_test).round(3)],
'MSE' : [mean_squared_error(ridgemod.predict(X1_test),y1_test).round(2)],
'MAE' : [mean_absolute_error(ridgemod.predict(X1_test),y1_test)]})



## Lasso Regression

Lasso Model: 
- $R^2= 0.723$
- $MSE= 7131$	
- $MAE= 53.33$


In [ ]:
from sklearn.linear_model import Lasso, LassoCV

In [ ]:
ols_bfat1 = Lasso(alpha=0,normalize=True)
ols_bfat1.fit(X1_train, y1_train)

In [ ]:
mean_squared_error(ols_bfat1.predict(X1_test),y1_test).round(2)

In [ ]:
Lassocv = LassoCV(alphas = lambdas, normalize = True,cv=5)
Lassocv.fit(X1_train, y1_train)
Lassocv.alpha_

In [ ]:
Lassomod = Lasso(alpha = Lassocv.alpha_, normalize = True)
Lassomod.fit(X1_train, y1_train)

In [ ]:
LassoCV_MSE= mean_squared_error(Lassomod.predict(X1_test),y1_test).round(2)
LassoCV_R2=Lassomod.score(X1_test,y1_test).round(3)
LassoCV_MAE = mean_absolute_error(Lassomod.predict(X1_test),y1_test)
print('R^2:', LassoCV_R2)
print('MSE:', LassoCV_MSE)
print('MAE:', LassoCV_MAE)

In [ ]:
LassoCV_Results=pd.DataFrame({'Models':['LassoCV'],
'MSE': [mean_squared_error(Lassomod.predict(X1_test),y1_test).round(2)],
'R2': [Lassomod.score(X1_test,y1_test).round(3)],
'MAE' : [mean_absolute_error(Lassomod.predict(X1_test),y1_test)]})

## Bagging 

Bagging Model: 
- $R^2 = 0.719$
- $MSE = 7243$
- $MAE = 49.51$

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
num_trees=np.linspace(50,2000,10)

In [ ]:
oob_res=[]
for trs in num_trees:
    res = BaggingRegressor(base_estimator=DecisionTreeRegressor(),n_estimators=int(trs),oob_score=True,random_state=5).fit(X1_train, y1_train)
    oob_res.append(res.oob_score_)

In [ ]:
plt.plot(num_trees,oob_res)

In [ ]:
num_trees[np.argmax(oob_res)]

In [ ]:
opt_trs=int(num_trees[np.argmax(oob_res)])
opt_trs

In [ ]:
bagging = BaggingRegressor(base_estimator=DecisionTreeRegressor(),n_estimators=opt_trs,random_state=5).fit(X1_train, y1_train)

In [ ]:
baggingCV_R2=bagging.score(X1_test,y1_test).round(3)
baggingCV_MSE= mean_squared_error(bagging.predict(X1_test),y1_test).round(2)
baggingCV_MAE = mean_absolute_error(bagging.predict(X1_test),y1_test)
print('R^2:', baggingCV_R2)
print('MSE:', baggingCV_MSE)
print('MAE:', baggingCV_MAE)


In [ ]:
baggingCV_results= pd.DataFrame({'Models':['Bagging_CV'],
'R2':[bagging.score(X1_test,y1_test).round(3)],
'MSE': [mean_squared_error(bagging.predict(X1_test),y1_test).round(3)],
'MAE': [mean_absolute_error(bagging.predict(X1_test),y1_test).round(3)]})

## Random Forest Model

Random Forest Model:
- $R^2: 0.732$
- $MSE: 6899.98$
- $MAE: 0.732$

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=266,max_features='sqrt',random_state=5).fit(X1_train,y1_train)

In [ ]:
RandomFR_R2=rf.score(X1_test,y1_test)
RandomFR_MSE= mean_squared_error(rf.predict(X1_test),y1_test).round(2)
RandomFRCV_MAE = mean_absolute_error(rf.predict(X1_test),y1_test)
print('R^2:', RandomFR_R2)
print('MSE:',RandomFR_MSE)
print('MAE:', RandomFR_R2)

In [ ]:
RandomFR_results=pd.DataFrame({'Models':['RandomForestRegressor'],
'R2':[rf.score(X1_test,y1_test).round(3)],
'MSE': [mean_squared_error(rf.predict(X1_test),y1_test).round(3)],
'MAE' : [mean_absolute_error(rf.predict(X1_test),y1_test).round(3)]})

## Boosting

Boosting Model: $R^2=0.598$


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'learning_rate':[0.1], 'n_estimators':[266], 'max_depth':[1,2,3,4,5]}

In [ ]:
bmod = GradientBoostingRegressor(random_state=5)

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,test_size=0.3, random_state=5)

In [ ]:
boost_regr = GridSearchCV(bmod, parameters)
boost_regr.fit(X1_train, y1_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.DataFrame.from_dict(boost_regr.cv_results_)
df.head()

In [ ]:
df[df['mean_test_score']==df['mean_test_score'].max()]

In [ ]:
boost =GradientBoostingRegressor(random_state=5,learning_rate=0.1,n_estimators=266,max_depth=2).fit(X1_train,y1_train)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
GBR_R2=boost.score(X1_test,y1_test)
GBR_MSE=mean_squared_error(boost.predict(X1_test),y1_test).round(3)
GBR_MAE= mean_absolute_error(boost.predict(X1_test),y1_test).round(2)
print('GBR_R2:', GBR_R2)
print('GBR_MSE:', GBR_MSE)
print('GBR_MAE:', GBR_MAE)

In [ ]:
RandomFR_R2=rf.score(X1_test,y1_test)
RandomFR_MSE= mean_squared_error(rf.predict(X1_test),y1_test).round(2)

In [ ]:
plt.figure(figsize=(8,6))
nfeat=X.shape[1]
plt.barh(np.arange(nfeat),boost.feature_importances_,align='center')
plt.yticks(np.arange(nfeat),list(X.columns))
plt.ylim(-1,nfeat)
plt.show()

In [ ]:
Gradient_Boosting_results= pd.DataFrame({'Models':['Gradieent_Boosting'],
'R2':[boost.score(X_test,y_test).round(3)],
'MSE': [mean_squared_error(boost.predict(X1_test),y1_test).round(3)],
'MAE' : [mean_absolute_error(boost.predict(X1_test),y1_test).round(3)]})

In [ ]:
Results= pd.concat([Liner_Regression_results, Ridge_Regression_results, LassoCV_Results, RandomFR_results,Gradient_Boosting_results,baggingCV_results], axis=0, ignore_index=True)
Results

In [ ]:
from matplotlib import rcParams

sns.catplot(
    data=Results, kind="bar",
    x="Models", y="R2", hue="Models",
    ci="sd", palette="dark", alpha=.6, height=6, dodge=False,size=6,aspect=2)

# EDA

In [ ]:
from numpy.lib.function_base import median
from numpy.ma.core import mean
Altpub=data[data['Published Nightly Rate'] <= 500].dropna()
Altpub.shape

In [ ]:
import seaborn as sns
sns.displot(Altpub['Published Nightly Rate'], bins=82, kde=True)

Distribution of the Published nightly rate and the mean published nightly rate is around 150

In [ ]:
Altpub['Neighborhood'].unique

In [ ]:
new=Altpub[Altpub['Neighborhood'].isin(['Harlem','Williamsburg','Brooklyn','Midtown','East Village'])]

In [ ]:
Plot=sns.violinplot(data=new, x='Neighborhood', y='Published Nightly Rate')
Plot.set_title('Distribution of Publised Nighly Rate prices for each neighberhood_group')

The violin plot shows the publised rate of the top 5 negibourhood(by count)

In [ ]:
Plot2=sns.catplot(x='Property Type', hue='Neighborhood', data=new, kind='count',height=8.27, aspect=11.7/8.27)
Plot2.set_xticklabels(rotation=90)

The barchart shows that majority of the property listed on the airbnb in the top 5 neighbourhood in New York majorly rent apartment

In [ ]:
b=Night_data.corr(method='kendall')
sns.heatmap(b)


Heat map of all the variables

In [ ]:
Corr=Night_data.corr()

In [ ]:
x=Corr.iloc[:,[14]]
sns.heatmap(x)

Heat map of correlation of Published Nightly Rate

In [ ]:
from wordcloud import WordCloud

In [ ]:
data18=data.dropna()
plt.subplots(figsize=(15,8))
wordcloud = WordCloud( background_color='white',width=1920,height=1080).generate(" ".join(data18['Neighborhood']))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('neighbourhood.png')
plt.show()

The world cloud shows that Harlem, East Side, Williamsburg, Midtown and Brooklyn are the most recurring words in the neighbourhood coloum.